In [2]:
# Tagging sentences using the default tagger
!nohup python3 tagger.py --files ../../2023-01-04/proof-corpus/sent01.tsv --cores 25 --output tagged_sents/tagged_sentences_test.txt

nohup: ignoring input and appending output to 'nohup.out'


In [3]:
# Create word bins for optimal tagger
!nohup python3 extract_sents.py --file tagged_sents/tagged_sentences_test.txt --word_file word_bin_list.txt --cores 20 --unique

nohup: ignoring input and appending output to 'nohup.out'


In [10]:
# Make testing and training bins
!nohup python3 make_test_train.py --num_train_sents 5 --num_test_bins 5 --train_extension test --test_extension test --word_list word_lists/nnp_verb_list_optimal.txt

nohup: ignoring input and appending output to 'nohup.out'


In [4]:
# Run main experiment
!nohup python3 main_experiment.py --train training_set/test.txt --test testing_set/test.txt --cores 25 --num_trials 5 --train_num_list 1,5 --iter_num_list 5 --extension test --wsj_test --dump --default_results 


nohup: ignoring input and appending output to 'nohup.out'


In [39]:
# Read experiment results
!python3 read_experiment_results.py --files experiments/*test* --output experiments/summary_test.txt


In [10]:
# Make new testing bin based on training bin
!nohup python3 make_test_train.py --train training_set/test.txt --num_test_bins 1 --test_extension test_2 --word_list word_lists/nnp_verb_list_partition.txt


nohup: ignoring input and appending output to 'nohup.out'


In [2]:
# Run dumped experiment
!nohup python3 dumped_main_experiment.py --tagger tagger/5sents_5iters_test_trial1.pk --test testing_set/refer_handtagged.txt --extension refer_test -p 25 --tag_n 3

116.98s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


nohup: ignoring input and appending output to 'nohup.out'


In [3]:
# Compare tagger weights in 7_5/5sents_5iters_7_5_trial
!python3 compare_tagger_weights.py --file disagreeing_sents/disagreeing_sents_7_8.txt --output compare_weights_test.txt

232.10s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


loaded
